- try pca20

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.metrics import log_loss
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.model_selection import StratifiedKFold, KFold
pd.set_option("max_columns", 1000)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  import sys


In [2]:
DIR = "/kaggle/input/lish-moa/"
train_feat = pd.read_csv(DIR+"train_features.csv")
test_feat = pd.read_csv(DIR+"test_features.csv")
#train_nonscore = pd.read_csv(DIR+"train_targets_nonscored.csv")
train_score = pd.read_csv(DIR+"train_targets_scored.csv")
sub = pd.read_csv(DIR+"sample_submission.csv")

In [3]:
target_feats = [ i for i in train_score.columns if i != "sig_id"]
g_feats = [i for i in train_feat.columns if "g-" in i]
c_feats = [i for i in train_feat.columns if "c-" in i]

# preprocess

In [4]:
def g_calc(row):
    return len(np.unique(row[g_feats].values))

tqdm_notebook.pandas(desc="progress")
train_feat["g_unique_num"] = train_feat.progress_apply(g_calc, axis=1)
test_feat["g_unique_num"] = test_feat.progress_apply(g_calc, axis=1)

categoricals = ["cp_dose", "cp_type"]

for f in categoricals:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_feat[f]))
    train_feat[f] = lbl.transform(list(train_feat[f]))
    test_feat[f] = lbl.transform(list(test_feat[f])) 
    
num = 20
pca_c_cols = ["pca-c"+str(i+1) for i in range(num)]
pca = PCA(n_components=num)
tmp_train = pca.fit_transform(train_feat[c_feats])
tmp_test = pca.transform(test_feat[c_feats])
tmp_train = pd.DataFrame(tmp_train, columns=pca_c_cols)
tmp_test = pd.DataFrame(tmp_test, columns=pca_c_cols)

train_feat = pd.concat([train_feat, tmp_train],axis=1)
test_feat = pd.concat([test_feat, tmp_test],axis=1)

train_feat.drop(c_feats, axis=1, inplace=True)
test_feat.drop(c_feats, axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# modelling

In [5]:
lgbm_params = {'objective': 'binary', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.01, "num_leaves": 10, 'random_seed':44, 'max_depth': 5}

def modelling_lgb(new_train, target_train, new_test, target):
    
    X_train = new_train.drop(['sig_id'],axis=1).copy()
    y_train = target_train[target].copy()
    X_test = new_test.copy()
    X_test = new_test.drop(['sig_id'],axis=1).copy()
        
    pred_value = np.zeros(X_test.shape[0])

    #remove_features = []
    #for i in X_train.columns:
    #    if (X_train[i].std() == 0) and i not in remove_features:
    #        remove_features.append(i)
    #X_train = X_train.drop(remove_features, axis=1)
    #X_test = X_test.drop(remove_features, axis=1)
    
    features_list = [i for i in X_train.columns]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    
    n_folds=4
    if target not in ["erbb2_inhibitor", "atp-sensitive_potassium_channel_antagonist"]:
        skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    else:
        skf=KFold(n_splits = n_folds, shuffle=True, random_state=0)

    models = []

    valid = np.zeros([X_train.shape[0]])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_valid2 = X_train.iloc[test_index,:]
        y_valid2 = y_train.iloc[test_index]
        
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_valid2, y_valid2, reference=lgb_train)
        
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval], 
               num_boost_round=10000,early_stopping_rounds=50,verbose_eval = 1000) 

        valid_predict = clf.predict(X_valid2, num_iteration = clf.best_iteration)
        valid[test_index] = valid_predict
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
        models.append(clf)
        
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
    feature_importance_df = feature_importance_df.sort_values("Average", ascending=False).reset_index(drop=True)

    score = log_loss(y_train, valid)
    print("oof log_loss= {} ".format(score))
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    
    return valid, pred_value, feature_importance_df,score

train_checkscore = train_score.copy()

target_list = []
log_loss_list = []

for ind, target in enumerate(target_feats):
    print(ind, target)
    valid, pred_value, feature_importance_df, score = modelling_lgb(train_feat, train_score, test_feat, target)
    train_checkscore[target] = valid
    target_list.append(target)
    log_loss_list.append(score)
    print("top10: ", list(feature_importance_df.iloc[:10]["Feature"]))
    print("last10: ", list(feature_importance_df.iloc[-10:]["Feature"]))
    del feature_importance_df
    sub[target] = pred_value

0 5-alpha_reductase_inhibitor
Fold 1
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's binary_logloss: 0.00528337	valid_1's binary_logloss: 0.00559136
Fold 2
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[88]	training's binary_logloss: 0.000812325	valid_1's binary_logloss: 0.00645328
Fold 3
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[87]	training's binary_logloss: 0.000850072	valid_1's binary_logloss: 0.00529427
Fold 4
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's binary_logloss: 0.00527729	valid_1's binary_logloss: 0.00559489
oof log_loss= 0.005733473994181422 
top10:  ['g-211', 'g-278', 'g-679', 'g-235', 'g-79', 'g-364', 'g-719', 'g-134', 'g-351', 'g-46']
last10:  ['g-299', 'g-311', 'g-301', 'g-302', 'g-304', 'g-305', 'g-306', 'g-308', 'g-309', 'g-395']
1 11-

In [6]:
# local score
scores = []
for target_col in target_feats:
    scores.append(log_loss(train_score[target_col], train_checkscore[target_col]))
print(np.mean(scores))

0.01611245476807251


In [7]:
difficult_list = pd.DataFrame(target_list, columns=["Target"])
difficult_list["score"] = log_loss_list
np.transpose(difficult_list)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
Target,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_i

In [8]:
sub.to_csv("submission.csv", index=False)